<a href="https://colab.research.google.com/github/ohnoccf/F1-score/blob/main/250521_F1_score_calculation_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In Google Colab, create 3 folders (1, 2, 3) and upload pairs (ground truth, prediction) of images in folder1 (ground truth) and folder2 (prediction).
# Run the code and specify the path of each folder (1, 2, 3).
# The results are saved to the new csv file in the output folder (folder3)

import os
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def load_image_as_binary(file_path, threshold=127):
      image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
      if image is None:
        raise ValueError(f"Unable to load image: {file_path}")
      _, binary_image = cv2.threshold(image, threshold, 1, cv2.THRESH_BINARY)
      return binary_image

def compute_metrics(ground_truth, prediction):
    gt_flat = ground_truth.flatten()
    pred_flat = prediction.flatten()

    acc = accuracy_score(gt_flat, pred_flat)
    prec = precision_score(gt_flat, pred_flat, zero_division=0)
    rec = recall_score(gt_flat, pred_flat, zero_division=0)
    f1 = f1_score(gt_flat, pred_flat, zero_division=0)
    return acc, prec, rec, f1

def main():
    print("Please enter the following folder paths:")
    input_folder1 = input("Enter path to first input (ground truth) folder: ").strip()
    input_folder2 = input("Enter path to second input (prediction) folder: ").strip()
    output_folder = input("Enter path to output folder: ").strip()

    if not os.path.exists(output_folder):
      os.makedirs(output_folder)
      print(f"Created output folder: {output_folder}")

    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')

    images1 = sorted([f for f in os.listdir(input_folder1) if f.lower().endswith(valid_extensions)])
    images2 = sorted([f for f in os.listdir(input_folder2) if f.lower().endswith(valid_extensions)])

    print(f"\nFound {len(images1)} valid images in '{input_folder1}'")
    print(f"Found {len(images2)} valid images in '{input_folder2}'")

    if len(images1) == 0 or len(images2) == 0:
      print("Error: One of the input folders does not contain any valid images.")
      return

    if len(images1) != len(images2):
      print("Error: The number of images in the two folders does not match.")
      return

    results = []

    for file1, file2 in zip(images1, images2):
        path1 = os.path.join(input_folder1, file1)
        path2 = os.path.join(input_folder2, file2)
        print(f"\nProcessing pair: '{file1}' and '{file2}'")
        try:
            img1 = load_image_as_binary(path1)
            img2 = load_image_as_binary(path2)
        except Exception as e:
            print(f"Error: {e}")
            continue

        if img1.shape != img2.shape:
            print(f"Skipping pair '{file1}' and '{file2}': image dimensions do not match ({img1.shape} vs {img2.shape}).")
            continue

        accuracy, precision, recall, f1 = compute_metrics(img1, img2)
        print(f"Metrics for '{file1}' and '{file2}': Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

        results.append({
            "Image1": file1,
            "Image2": file2,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1
        })

    if results:
        output_csv = os.path.join(output_folder, "evaluation_metrics.csv")
        df = pd.DataFrame(results)
        df.to_csv(output_csv, index=False)
        print(f"\nAll metrics have been saved to: {output_csv}")
    else:
        print("\nNo valid image pairs were processed. CSV file was not created.")

if __name__ == '__main__':
    main()
